In [76]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

pd.set_option("display.max_columns", None)
# pd.
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Reading data

In [77]:
print(len(train_df[train_df.target == 1]))
print(len(train_df[train_df.target == 0]))
# Balanced data

In [78]:
train_df = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/train.csv")
test_df = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/test.csv")
sub = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/sample_submission.csv")

In [79]:
print(len(train_df))

In [80]:
train_df.head(5)

In [81]:
Summary = pd.DataFrame(train_df.dtypes, columns = ['Dtype'])
Summary['null'] = train_df.isnull().sum()
Summary['max'] = train_df.max()
Summary['min'] = train_df.min()
Summary['first'] = train_df.iloc[0, :]
Summary['second'] = train_df.iloc[1, :]
Summary['third'] = train_df.iloc[2, :]

In [82]:
Summary

In [83]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc

In [84]:
y = train_df["target"]
train_df = train_df.drop(["target", "id"], axis = 1)

In [85]:
test_df = test_df.drop("id", axis = 1)

In [86]:
# train_df.columns == test_df.columns

In [87]:
X_train, X_test, y_train, y_test = train_test_split(train_df,y, test_size = 0.25, stratify = y, random_state = 123)

In [88]:
len(X_train)
len(X_test)

In [89]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [90]:
lr = LogisticRegression(random_state=123, C = 0.01, penalty = 'l2')
lr.fit(X_train, y_train)

In [91]:
y_pred = lr.predict_proba(X_test)
y_pred = y_pred[:, 1]

In [92]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
print(auc(fpr, tpr))
# The score is without adding features, 0.7469048481960304

In [93]:
test_df = scaler.transform(test_df)

In [94]:
y_res = lr.predict_proba(test_df)
y_res = y_res[:, 1]

In [95]:
sub["target"] = y_res
sub.to_csv("lr.csv", index=False)
sub.head(10)